In [1]:
from tinygrad import Tensor
from tinygrad.shape.shapetracker import ShapeTracker
from tinygrad.shape.view import View

In [5]:
# dot product between [1,2] and [3,4] is 1 * 3 + 2 * 4 = 11
b = Tensor([1,2])
b = Tensor([3,4])
res = b.dot(b)
print(res.numpy()) # 11

*** CUDA       5 copy        8,    CUDA <- NPY          arg   2 mem  0.00 GB tm   1381.20us/     6.38ms (    0.00 GFLOPS,    0.00 GB/s) 
*** CUDA       6 copy        8,    CUDA <- NPY          arg   2 mem  0.00 GB tm    336.60us/     6.72ms (    0.00 GFLOPS,    0.00 GB/s) 
*** CUDA       7 r_2                                    arg   3 mem  0.00 GB tm    623.17us/     7.34ms (    0.00 GFLOPS,    0.00 GB/s) ['dot']
*** CLANG      8 copy        4,   CLANG <- CUDA         arg   2 mem  0.00 GB tm    208.30us/     7.55ms (    0.00 GFLOPS,    0.00 GB/s) 
11


In [77]:
print("--------------------A")
t = Tensor(list(range(20)))
# b = Tensor(list(range(20)))

print(f"memory:\n {t.numpy()}\n")

a = t[2:14].reshape(3, 4).pad((None, (0, 3)))#.shrink((None, (0, 4)))
print(f"tensor:\n {a.numpy()}\n")

print(f'{a.shape=}')
print(f'{a.lazydata.st.views=}')
idx, valid = a.lazydata.st.expr_idxs()
print(f'{idx=}')  # index -> buf location
print(f'{valid=}')  # mask

print(a.lazydata.st.views[0])
# print(a.lazydata.st.views[1])

v_1 = a.lazydata.st.views[0]
print(v_1)

a.reshape((1, 21))
print(a.lazydata.st.views)

# why no new view created? of view didn't update?

--------------------A
memory:
 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]

tensor:
 [[ 2  3  4  5  0  0  0]
 [ 6  7  8  9  0  0  0]
 [10 11 12 13  0  0  0]]

a.shape=(3, 7)
a.lazydata.st.views=(View(shape=(3, 7), strides=(4, 1), offset=2, mask=((0, 3), (0, 4)), contiguous=False),)
idx=((Variable('idx0', 0, 2)*4)+NumNode(2)+Variable('idx1', 0, 6))
valid=(Variable('idx1', 0, 6)<4)
View(shape=(3, 7), strides=(4, 1), offset=2, mask=((0, 3), (0, 4)), contiguous=False)
View(shape=(3, 7), strides=(4, 1), offset=2, mask=((0, 3), (0, 4)), contiguous=False)
(View(shape=(3, 7), strides=(4, 1), offset=2, mask=((0, 3), (0, 4)), contiguous=False),)


In [78]:
print("--------------------B")
b = Tensor(list(range(20)))
print("b: ", b)
print("b.numpy():", b.numpy())
print("b.lazydata.st.views: ", b.lazydata.st.views)

--------------------B
b:  <Tensor <LB CUDA (20,) int (<MetaOps.COPY: 3>, None)> on CUDA with grad None>
b.numpy(): [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
b.lazydata.st.views:  (View(shape=(20,), strides=(1,), offset=0, mask=None, contiguous=True),)


In [79]:
print("--------------------C")
c = Tensor.eye(3).realize()
print(c.numpy())

# assert not a.lazydata.is_unrealized_const()

# print("b.realize: ", b.realize())

# print("b.lazydata.base: ", b.lazydata.base)
print()

print(f'{c.shape=}')
print(f'{c.lazydata.st.views=}')
idx, valid = c.lazydata.st.expr_idxs()
print(f'{idx=}')  # index -> buf location
print(f'{valid=}')  # mask

print(c.lazydata.st.views[0])
print(c.lazydata.st.views[1])

# why are there two views here?

--------------------C
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]

c.shape=(3, 3)
c.lazydata.st.views=(View(shape=(3, 4), strides=(0, 0), offset=0, mask=((0, 3), (0, 1)), contiguous=False), View(shape=(3, 3), strides=(3, 1), offset=0, mask=None, contiguous=True))
idx=NumNode(0)
valid=(((((Variable('idx0', 0, 2)*3)%4)+Variable('idx1', 0, 2))%4)<1)
View(shape=(3, 4), strides=(0, 0), offset=0, mask=((0, 3), (0, 1)), contiguous=False)
View(shape=(3, 3), strides=(3, 1), offset=0, mask=None, contiguous=True)


In [ ]:
print("--------------------C")

s = ShapeTracker(views=(
    # idk random
    View.create(shape=(20,), strides=(1,), offset=0, mask=None),
    View.create(shape=(3, 5), strides=(3, 1), offset=2, mask=((0, 3), (0, 3))),
))
print(s)
print(s.simplify())

v = View.create(shape=(9,), strides=(1,), offset=0, mask=None)
print(v.reshape((3, 3)))

In [7]:
# found a 3 view shapetracker that can be expressed with 2 views.

View(shape=(1, 1, 2, 4, 2, 4), strides=(0, 0, 2, 8, 1, 4), offset=0, mask=((0, 1), (0, 1), (0, 2), (0, 2), (0, 2), (0, 2)), contiguous=False)
View(shape=(1, 1, 9, 9), strides=(0, 0, 8, 1), offset=0, mask=((0, 1), (0, 1), (0, 8), (0, 8)), contiguous=False)
View(shape=(1, 1, 1, 1, 3, 3, 3, 3), strides=(0, 0, 0, 0, 27, 9, 3, 1), offset=0, mask=None, contiguous=True)


# the trick is that you can have extra mask on the final view because it's already masked on the top one

View(shape=(1, 1, 2, 4, 2, 4), strides=(0, 0, 2, 8, 1, 4), offset=0, mask=((0, 1), (0, 1), (0, 2), (0, 2), (0, 2), (0, 2)), contiguous=False)
View(shape=(1, 1, 1, 1, 3, 3, 3, 3), strides=(0, 0, 0, 0, 24, 8, 3, 1), offset=0, mask=((0, 1), (0, 1), (0, 1), (0, 1), (0, 2), (0, 3), (0, 2), (0, 3)), contiguous=False)

View(shape=(1, 1, 1, 1, 3, 3, 3, 3), strides=(0, 0, 0, 0, 27, 9, 3, 1), offset=0, mask=None, contiguous=True)

In [5]:
memory = Tensor.arange(start=0, stop=64, step=1)
print(f"memory:\n{memory.numpy()}\n")


t1 = Tensor.arange(start=0, stop=64, step=1)
t1_shaped = t1.reshape((1, 1, 2, 4, 2, 4))
print(f"t1_shaped:\n{t1_shaped.numpy()}\n")
print(f"t1_shaped.shapetracker: {t1_shaped.lazydata.st} \n")


t1.lazydata.st.stride((1, 1, 2, 8, 1, 4))
print(f"t1_shaped.shapetracker: {t1_shaped.lazydata.st}\n")

# how can I change the strides to match the views above? .stride() does not allow 0 in any stride dimension for some reason.

# print(f"t1:\n{t1.realize().numpy()}\n")



# t2 = Tensor.arange(start=0, stop=81, step=1)
# t2_shaped = t2.reshape((1, 1, 9, 9))
# print(f"t2_shaped:\n{t2_shaped.numpy()}\n")

# t3 = Tensor.arange(start=0, stop=81, step=1)
# t3_shaped = t3.reshape((1, 1, 1, 1, 3, 3, 3, 3))
# print(f"t3_shaped:\n{t3_shaped.numpy()}\n")

memory:
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63]

t1_shaped:
[[[[[[ 0  1  2  3]
     [ 4  5  6  7]]

    [[ 8  9 10 11]
     [12 13 14 15]]

    [[16 17 18 19]
     [20 21 22 23]]

    [[24 25 26 27]
     [28 29 30 31]]]


   [[[32 33 34 35]
     [36 37 38 39]]

    [[40 41 42 43]
     [44 45 46 47]]

    [[48 49 50 51]
     [52 53 54 55]]

    [[56 57 58 59]
     [60 61 62 63]]]]]]

t1_shaped.shapetracker: ShapeTracker(views=(View(shape=(1, 1, 2, 4, 2, 4), strides=(0, 0, 32, 8, 4, 1), offset=0, mask=None, contiguous=True),)) 

t1_shaped.shapetracker: ShapeTracker(views=(View(shape=(1, 1, 2, 4, 2, 4), strides=(0, 0, 32, 8, 4, 1), offset=0, mask=None, contiguous=True),))

